# VOTT \*.json to annotations

In [3]:
import numpy as np
import time
import sys
import os
import random
# from skimage import io
import pandas as pd
# from matplotlib import pyplot as plt
from shutil import copyfile

# import cv2
# import tensorflow as tf

import json
from pprint import pprint


In [4]:
base_path = '../faster-rcnn_vgg16/Dataset/airport'

In [5]:
df = pd.DataFrame(columns=['data_file_json', 'FileName', 'XMin', 'XMax', 'YMin', 'YMax', 'ClassName', 'Width','Height'])


for data_file_json in ['../data/labeling/data-merged.json','../data/labeling/data-filtered-yolo.json']:

    with open(data_file_json) as f:
        data = json.load(f)
    print("in {} found # keys: {}".format(data_file_json, len(data["frames"].keys())))
    
    i=1
    imax = len(data["frames"])

    for key,val in data["frames"].items():

        sys.stdout.write('Parse train_imgs ' + str(i) + '; Number of images: ' + str(len(data["frames"])) + '\r')
        sys.stdout.flush()

        for v in val:
            try:
                df = df.append({'data_file_json': data_file_json,
                                                    'FileName': key, 
                                                    'XMin': v["x1"], 
                                                    'XMax': v["x2"], 
                                                    'YMin': v["y1"], 
                                                    'YMax': v["y2"], 
                                                    'ClassName': v["tags"][0],
                                                    'Width': v["width"],
                                                    'Height': v["height"],}, 
                                                   ignore_index=True)
            except:
                continue
        i+=1

sys.stdout.write("\nFinal dataset has {} images".format(len(df)))

in ../data/labeling/data-merged.json found # keys: 339
in ../data/labeling/data-filtered-yolo.json found # keys: 449
Parse train_imgs 449; Number of images: 449
Final dataset has 1609 images

In [6]:
df.groupby("data_file_json").FileName.nunique()

data_file_json
../data/labeling/data-filtered-yolo.json    210
../data/labeling/data-merged.json           244
Name: FileName, dtype: int64

In [8]:
df_sorted = df.sort_values(by=['FileName'])
df_sorted.head(10)

,data_file_json,FileName,XMin,XMax,YMin,YMax,ClassName,Width,Height
963,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00000_at_00-00-01.jpg,247.0,279.0,469.0,562.0,person,1280,720
1295,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,205.0,259.0,522.0,662.0,person,1280,720
1296,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00001_at_00-00-01.jpg,247.0,278.0,469.0,567.0,person,1280,720
994,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,247.0,276.0,469.0,566.0,person,1280,720
993,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00002_at_00-00-01.jpg,211.0,259.0,509.0,642.0,person,1280,720
995,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00003_at_00-00-01.jpg,232.0,280.0,469.0,622.0,person,1280,720
1241,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00004_at_00-00-01.jpg,234.0,279.0,471.0,614.0,person,1280,720
1017,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00005_at_00-00-01.jpg,234.0,279.0,471.0,614.0,person,1280,720
927,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00006_at_00-00-01.jpg,250.0,279.0,469.0,571.0,person,1280,720
926,../data/labeling/data-filtered-yolo.json,videoplayback-720p-short_00006_at_00-00-01.jpg,210.0,262.0,513.0,622.0,person,1280,720


### Write  merged json

In [24]:
x = {}

for idx, row in df_sorted.iterrows():
    x1 = int(row['XMin'])
    x2 = int(row['XMax'])
    y1 = int(row['YMin'])
    y2 = int(row['YMax'])

#     res_file_path = base_path + '/' + t
#     fileName = os.path.join(res_file_path, row['FileName'])
    className = row['ClassName']
    x[row['FileName']] = x.get(row['FileName'],[])
    x[row['FileName']].append({
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
                "width": int(row['Width']),
                "height": int(row['Height']),
                "box": {
                    "x1": x1,
                    "y1": y1,
                    "x2": x2,
                    "y2": y2
                },
                "points": [
                    {
                        "x": x1,
                        "y": y1
                    },
                    {
                        "x": x2,
                        "y": y1
                    },
                    {
                        "x": x2,
                        "y": y2
                    },
                    {
                        "x": x1,
                        "y": y2
                    }
                ],
                "UID": "",
                "id": -1,
                "type": "rect",
                "tags": [
                    row['ClassName']
                ],
                "name": 2
            })
    
    
# annotation_file = base_path + "/" + t + "_annotation.txt"
# f= open(annotation_file,"w+")
# f.write(fileName + ',' + str(x1) + ',' + str(y1) + ',' + str(x2) + ',' + str(y2) + ',' + className + '\n')
# f.close()
# print("Output for {}: {}".format(t,annotation_file))

In [25]:
out = {}
out["frames"] = x
out["framerate"] = "1"
out["inputTags"]= "cisterna,letadlo,schody,schody1,person,truck"
out["suggestiontype"]="track"
out["scd"]= "false"
out["visitedFrames"]= []
out["tag_colors"]= [
    "#ff0ec7",
    "#0fffae",
    "#c88d0d",
    "#0046fc"
    "#ff0ec7",
    "#ffffae",
    "#c8000d",
    "#2222fc"
]

In [29]:
with open('data-filtered-merged-yolo.json', 'w') as outfile:  
    json.dump(out, outfile)